In [ ]:
import json
import os
from library_dicom.dicom_processor.tools.folders import *
from library_dicom.dicom_processor.tools.create_mip import *
from library_dicom.dicom_processor.tools.threshold_mask import *
import numpy as np
import SimpleITK as sitk
import csv
import shutil

### Script to write csv_file with every nifti for DeepOncology repertory

#### - Read JSON dataset 

In [ ]:
json_path = 'abs//path//json_dataset saved in by nifti_builder_from_json.ipynb'
dataset = []
with open(json_path) as json_file : 
    reader = json.load(json_file)
    for info in reader :
        dataset.append(info)
print(len(dataset))

#### - Get the list of every nifti in the directory 

In [ ]:
nifti_directory = 'abs//path//nifti//directory'
liste = os.listdir(nifti_directory)
liste_nifti = []
for nifti in liste : 
    if 'nii' in nifti : 
        liste_nifti.append(os.path.join(nifti_directory, nifti))

In [ ]:
len(liste_nifti)

#### - Add nifti PT CT MASK for every data/study in dataset 

In [ ]:
for patient in dataset: 
    uid = patient[4]
    subliste = []
    for nifti in liste_nifti : 
        if uid in nifti : 
            subliste.append(nifti)
    subliste.sort()
    if len(subliste) == 3 :   #3 if PET, CT and MASK/ 2 if only PET and CT 
        for i in range(len(subliste)):
            patient.append(subliste[i])
#patient = [path_1, modal_1, path_2, modal_2, study_uid, type_, patient_id, csv_file, nifti_CT, nifti_PET, nifti_MASK]
    

In [ ]:
error = []
for serie in dataset : 
    if len(serie) != 11 : #11 if CSV and CT/PET/MASK ; 10 IF CT/PET/MASK ; 9 if ONLY CT/PET
        print(serie)
        error.append(serie)
for err in error : 
    dataset.remove(err)
print('dataset cleaned')

In [ ]:
print(" Number of study with complete nifti: ", len(dataset))

#### - Create folder with patient_id and put nifti into it 

In [ ]:
directory =  '/media/deeplearning/LACIE SHARE/PVAB/NIFTI/no_suv'
for serie in dataset: 
    #serie = [path_1, modal_1, path_2, modal_2, study_uid, type_, patient_id, csv_file,  nifti_CT, nifti_PET, nifti_MASK]
    patient_id = serie[6]
    pet_type = serie[5]
    study_uid = serie[4]
    nifti_ct = serie[-3]
    nifti_pet = serie[-2]
    nifti_mask = serie[-1]

    folder = directory+'/'+patient_id
    if not os.path.exists(folder):
        os.makedirs(folder)
        print(folder, 'created')
    else : print(folder, "Patient folder already created")

    subfolder = folder+'/'+pet_type
    if not os.path.exists(subfolder) :
        os.makedirs(subfolder)
        print(subfolder, 'created')
    else : print(subfolder, "type subfolder alreader created ! ", pet_type)

    new_ct_path = subfolder+'/'+nifti_ct.split('/')[-1]
    new_pet_path = subfolder+'/'+nifti_pet.split('/')[-1]
    new_mask_path = subfolder+'/'+nifti_mask.split('/')[-1]
    
    try : 
        os.rename(nifti_ct, new_ct_path)
        os.rename(nifti_pet, new_pet_path)
        os.rename(nifti_mask, new_mask_path)
        serie[-3] = new_ct_path
        serie[-2] = new_pet_path 
        serie[-1] = new_mask_path

    except Exception as err : 
        print(err)
    

#### - Write csv_file 

In [ ]:
directory = 'abs//path//directory'
filename = 'name of the CSV file.csv'

with open(os.path.join(directory, filename), 'w') as csv_file : 
    csv_writer = csv.writer(csv_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(["PATIENT_ID", "STUDY UID", "PET", "NIFTI_CT", "NIFTI_PET", "NIFTI_MASK", "STUDY"])
    for serie in dataset: 
        #serie = [path_1, modal_1, path_2, modal_2, study_uid, type_, patient_id, nifti_CT, nifti_PET, nifti_MASK]
        csv_writer.writerow([serie[6], serie[4], serie[5], serie[-3], serie[-2], serie[-1], 'name_study'])